In [6]:
class TransportationProblem(object):
    def __init__(self, N,weights):
        self.N = N
        self.weights=weights

    def startState(self):
        return 1

    def isEnd(self, state):
        return state == self.N

    def succAndCost(self, state):
        result = []
        if state + 1 <= self.N:
            result.append(('walk', state + 1, self.weights['walk']))
        if state + 3 <= self.N:
            result.append(('run', state + 3, self.weights['run']))
        if 2 * state <= self.N:
            result.append(('tram', 2 * state, self.weights['tram']))
        return result

In [7]:
def dynamicProgramming(problem):
    cache = {} # state -> futureCost(state), action, newState, cost
    def futureCost(state):
        # Base case
        if problem.isEnd(state):
            return 0
        if state in cache: # Exponential savings
            return cache[state][0]
        # Actually doing work
        result = min((cost+futureCost(newState), action, newState, cost) \
                for action, newState, cost in problem.succAndCost(state))
        cache[state] = result
        return result[0]

    state = problem.startState()
    totalCost = futureCost(state)

    # Recover history
    history = []
    while not problem.isEnd(state):
        _, action, newState, cost = cache[state]
        history.append((action, newState, cost))
        state = newState

    return (futureCost(problem.startState()), history)


In [12]:
def predict(N,weights):
    problem=TransportationProblem(N,weights)
    totalCost,history=dynamicProgramming(problem)
    return [action for action, newState ,cost in history]
def generateExamples():
    trueWeights={'walk':1,'run':1.5,'tram':3}
    return [(N,predict(N,trueWeights)) for N in range (1,21)]
    


# Printing Examples

In [13]:
examples=generateExamples()
print("Dataset is:")
for example in examples:
    print(' ',example)

Dataset is:
  (1, [])
  (2, ['walk'])
  (3, ['walk', 'walk'])
  (4, ['run'])
  (5, ['run', 'walk'])
  (6, ['run', 'walk', 'walk'])
  (7, ['run', 'run'])
  (8, ['run', 'run', 'walk'])
  (9, ['run', 'run', 'walk', 'walk'])
  (10, ['run', 'run', 'run'])
  (11, ['run', 'run', 'run', 'walk'])
  (12, ['run', 'run', 'run', 'walk', 'walk'])
  (13, ['run', 'run', 'run', 'run'])
  (14, ['run', 'run', 'tram'])
  (15, ['run', 'run', 'tram', 'walk'])
  (16, ['run', 'run', 'walk', 'tram'])
  (17, ['run', 'run', 'tram', 'run'])
  (18, ['run', 'run', 'walk', 'walk', 'tram'])
  (19, ['run', 'run', 'walk', 'tram', 'run'])
  (20, ['run', 'run', 'run', 'tram'])


# Structured Perceptron

In [14]:
def structuredPerceptron(examples):
    weights={'walk':0,'run':0,'tram':0}
    for t in range(100):
        numMistakes=0
        for N, trueActions in examples:
            predActions=predict(N,weights)
            if predActions != trueActions:
                numMistakes+=1
            for action in trueActions:
                weights[action]-=1
            for action in predActions:
                weights[action]+=1
        print('Iteration {},numMistakes={}, weights={}'.format(t,numMistakes,weights))
        if numMistakes==0:
            break;
    

In [15]:
structuredPerceptron(examples)

Iteration 0,numMistakes=9, weights={'walk': 19, 'run': 1, 'tram': 0}
Iteration 1,numMistakes=12, weights={'walk': 8, 'run': 6, 'tram': 7}
Iteration 2,numMistakes=6, weights={'walk': 4, 'run': 6, 'tram': 10}
Iteration 3,numMistakes=6, weights={'walk': 4, 'run': 6, 'tram': 11}
Iteration 4,numMistakes=2, weights={'walk': 5, 'run': 6, 'tram': 11}
Iteration 5,numMistakes=2, weights={'walk': 4, 'run': 7, 'tram': 11}
Iteration 6,numMistakes=4, weights={'walk': 4, 'run': 6, 'tram': 12}
Iteration 7,numMistakes=0, weights={'walk': 4, 'run': 6, 'tram': 12}
